# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import re
import pickle

nltk.download(['punkt', 'wordnet', 'omw-1.4', 'stopwords'])
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dalem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dalem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dalem\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dalem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql("SELECT * FROM messages", engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [3]:
# How many words are there?
counts = [len(word_tokenize(msg)) for msg in X]
print(f'Total words in dataset: {sum(counts)}')

Total words in dataset: 708271


In [4]:
Y.sum(axis=0)

related                   20093
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [5]:
# Remove child_alone as there is no data
Y = Y.drop(columns=['child_alone'])

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):

    # Make lowercase and remove non alphanumeric
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [tok for tok in tokens if tok not in stop_words]

    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok.strip())
        clean_tokens.append(clean_tok)

    return clean_tokens

In [7]:
# Test the tokenizer
for idx in range(10):
    print(df.message.iloc[idx])
    print(tokenize(df.message[idx]))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over
['hurricane']
Looking for someone but no name
['looking', 'someone', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']
Information about the National Palace-
['information', 'national', 'palace']
Storm at sacred heart of jesus
['storm', 'sacred', 'heart', 'jesus']
Please, we need tents and water. We are in Silo, Thank you!
['please', 'need', 'tent', 'water', 'silo', 'thank']
I would like to receive the messages, thank you
['would', 'like', 'receive', 'message', 'thank']
I am in Croix-des-Bouqu

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
# n_estimators set to 10. Default used to be 10 but was changed to 100 in
# sklearn v0.22 (Dec 2019). Might explain the long training time.
pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [10]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x00000245EC6E6550>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10,
                                                                        n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def report(Y_test, Y_pred):
    precision = []
    recall = []
    f1 = []
    support = []
    for col in Y_test:
        
        # Print full report for category
        report_str = classification_report(Y_test[col], Y_pred[col], zero_division=0)
        print(f'Class: {col}:\n{report_str}')

        # Store average values
        report = classification_report(Y_test[col], Y_pred[col], zero_division=0, output_dict=True)
        precision.append(report['weighted avg']['precision'])
        recall.append(report['weighted avg']['recall'])
        f1.append(report['weighted avg']['f1-score'])
        support.append(report['weighted avg']['support'])

    # Print the averages for everything
    print('Means of weighted averages:')
    print(f'Precision: {np.array(precision).mean()}, Recall: {np.array(recall).mean()}, f1-score: {np.array(f1).mean()} Support: {np.array(support).sum()}')


In [12]:
Y_pred = pipeline.predict(X_test)
Y_pred = pd.DataFrame(Y_pred, columns=Y.columns)
report(Y_test, Y_pred)

Class: related:
              precision    recall  f1-score   support

           0       0.63      0.46      0.54      1511
           1       0.85      0.92      0.88      5043

    accuracy                           0.81      6554
   macro avg       0.74      0.69      0.71      6554
weighted avg       0.80      0.81      0.80      6554

Class: request:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5393
           1       0.79      0.45      0.57      1161

    accuracy                           0.88      6554
   macro avg       0.84      0.71      0.75      6554
weighted avg       0.87      0.88      0.87      6554

Class: offer:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99     

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(tokenizer=<function tokenize at 0x00000245EC6E6550>)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10,
                                                          n_jobs=-1)))],
 'verbose': False,
 'tfidf': TfidfVectorizer(tokenizer=<function tokenize at 0x00000245EC6E6550>),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10,
                                                        n_jobs=-1)),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tf

In [14]:
# Pipeline
pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

# Test n_estimators and criterion
parameters = {
    'clf__estimator__n_estimators': [5, 10, 50],
    'clf__estimator__criterion': ['gini', 'entropy']
}

# create grid search object and train
model1 = GridSearchCV(pipeline, parameters, verbose=2)
model1.fit(X_train, Y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=5; total time=   7.0s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=5; total time=   7.0s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=5; total time=   7.1s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=5; total time=   7.0s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=5; total time=   7.2s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=10; total time=  10.2s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=10; total time=  10.4s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=10; total time=  10.7s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=10; total time=  11.2s
[CV] END clf__estimator__criterion=gini, clf__estimator__n_estimators=10; total time=  11.7s
[CV] END clf__e

GridSearchCV(estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x00000245EC6E6550>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))]),
             param_grid={'clf__estimator__criterion': ['gini', 'entropy'],
                         'clf__estimator__n_estimators': [5, 10, 50]},
             verbose=2)

In [15]:
# Compare scores
pd.DataFrame(model1.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__criterion,param_clf__estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.268389,0.073040,0.897984,0.042486,gini,5,"{'clf__estimator__criterion': 'gini', 'clf__es...",0.210272,0.208291,0.215921,0.201424,0.216938,0.210569,0.005623,6
1,9.907992,0.489879,1.029033,0.058131,gini,10,"{'clf__estimator__criterion': 'gini', 'clf__es...",0.234935,0.247711,0.243642,0.246439,0.240844,0.242714,0.004554,4
2,39.114987,4.560503,1.998456,0.292892,gini,50,"{'clf__estimator__criterion': 'gini', 'clf__es...",0.256039,0.274669,0.265005,0.267548,0.257121,0.264076,0.006901,1
3,7.363229,0.385406,0.929106,0.017345,entropy,5,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.204170,0.217192,0.214140,0.212360,0.209308,0.211434,0.004441,5
4,11.141211,0.131326,1.024041,0.046700,entropy,10,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.246885,0.251780,0.251526,0.241099,0.243388,0.246936,0.004271,3
5,45.009317,5.323179,1.951158,0.236351,entropy,50,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.254259,0.276195,0.263225,0.265005,0.254578,0.262652,0.008061,2


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
Y_pred = pd.DataFrame(model1.predict(X_test), columns=Y.columns)
report(Y_test, Y_pred)

Class: related:
              precision    recall  f1-score   support

           0       0.71      0.40      0.51      1511
           1       0.84      0.95      0.89      5043

    accuracy                           0.82      6554
   macro avg       0.78      0.67      0.70      6554
weighted avg       0.81      0.82      0.80      6554

Class: request:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5393
           1       0.82      0.50      0.62      1161

    accuracy                           0.89      6554
   macro avg       0.86      0.74      0.78      6554
weighted avg       0.89      0.89      0.88      6554

Class: offer:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# Try SGD classifier

pipeline = Pipeline([

    ('vect', TfidfVectorizer(tokenizer=None)),
    ('clf', MultiOutputClassifier(SGDClassifier(n_jobs=-1)))

])

# create grid search object and train
parameters = {}
model2 = GridSearchCV(pipeline, parameters, verbose=2)
model2.fit(X_train, Y_train)

# Test
Y_pred = pd.DataFrame(model2.predict(X_test), columns=Y.columns)

# Show results
report(Y_test, Y_pred)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.1s
[CV] END .................................................... total time=   1.0s
Class: related:
              precision    recall  f1-score   support

           0       0.75      0.39      0.51      1511
           1       0.84      0.96      0.90      5043

    accuracy                           0.83      6554
   macro avg       0.80      0.68      0.70      6554
weighted avg       0.82      0.83      0.81      6554

Class: request:
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5393
           1       0.82      0.59      0.68      1161

    accuracy 

In [18]:
# Try hashing vectoriser
pipeline = Pipeline([

    ('vect', HashingVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(SGDClassifier(n_jobs=-1)))

])

# create grid search object and train
parameters = {}
model3 = GridSearchCV(pipeline, parameters, verbose=2)
model3.fit(X_train, Y_train)

# Test
Y_pred = pd.DataFrame(model3.predict(X_test), columns=Y.columns)
report(Y_test, Y_pred)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\dalem\source\repos\udacity-nanoDS-notebooks\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END .................................................... total time=   5.3s


c:\Users\dalem\source\repos\udacity-nanoDS-notebooks\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END .................................................... total time=   5.3s


...

[CV] END .................................................... total time=   5.2s


c:\Users\dalem\source\repos\udacity-nanoDS-notebooks\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END .................................................... total time=   5.2s


c:\Users\dalem\source\repos\udacity-nanoDS-notebooks\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Class: related:
              precision    recall  f1-score   support

           0       0.74      0.44      0.55      1511
           1       0.85      0.95      0.90      5043

    accuracy                           0.83      6554
   macro avg       0.79      0.70      0.72      6554
weighted avg       0.82      0.83      0.82      6554

Class: request:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5393
           1       0.82      0.56      0.66      1161

    accuracy                           0.90      6554
   macro avg       0.86      0.77      0.80      6554
weighted avg       0.89      0.90      0.89      6554

Class: offer:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99     

In [20]:
# Optimie SGDClassifier

pipeline = Pipeline([

    ('vect', TfidfVectorizer(tokenizer=None)),
    ('clf', MultiOutputClassifier(SGDClassifier(n_jobs=-1)))

])

# create grid search object and train
parameters = {
    'vect__max_df': (0.25, 0.5, 0.75),
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__estimator__alpha': [1e-5, 1e-4, 1e-3],
}
model4 = GridSearchCV(pipeline, parameters, verbose=2)
model4.fit(X_train, Y_train)

# Test
Y_pred = pd.DataFrame(model4.predict(X_test), columns=Y.columns)

# Show results
report(Y_test, Y_pred)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 1); total time=   0.9s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 1); total time=   0.8s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 1); total time=   0.8s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 1); total time=   0.9s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 1); total time=   0.9s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 2); total time=   1.9s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 2); total time=   1.8s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 2); total time=   2.2s
[CV] END clf__estimator__alpha=1e-05, vect__max_df=0.25, vect__ngram_range=(1, 2); total time=   2.5s
[CV] END clf__estimat

In [99]:
# Compare scores
pd.DataFrame(model4.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__alpha,param_vect__max_df,param_vect__ngram_range,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.987113,0.053671,0.070060,0.002178,0.00001,0.25,"(1, 1)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.245614,0.247457,0.240336,0.254578,0.244914,0.246580,0.004635,18
1,1.965821,0.104365,0.161571,0.016697,0.00001,0.25,"(1, 2)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.263412,0.256358,0.260682,0.277976,0.266277,0.264941,0.007292,7
2,3.895934,0.316327,0.242271,0.021586,0.00001,0.25,"(1, 3)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.266972,0.254578,0.259919,0.273652,0.266531,0.264330,0.006531,8
3,1.068075,0.113591,0.086952,0.025963,0.00001,0.5,"(1, 1)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.248919,0.246185,0.241862,0.254069,0.248728,0.247953,0.003979,15
4,2.432980,0.206689,0.208937,0.020510,0.00001,0.5,"(1, 2)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.256547,0.252543,0.257375,0.276704,0.265514,0.261737,0.008588,10
5,4.749336,0.117678,0.314642,0.016034,0.00001,0.5,"(1, 3)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.257056,0.253052,0.257375,0.267803,0.262208,0.259499,0.005066,12
6,1.294007,0.051624,0.088396,0.009385,0.00001,0.75,"(1, 1)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.253242,0.245677,0.243133,0.257630,0.248474,0.249631,0.005219,14
7,2.797692,0.293645,0.206622,0.022842,0.00001,0.75,"(1, 2)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.263158,0.258138,0.257630,0.269837,0.266785,0.263110,0.004765,9
8,4.809908,0.093132,0.301508,0.012275,0.00001,0.75,"(1, 3)","{'clf__estimator__alpha': 1e-05, 'vect__max_df...",0.256801,0.250000,0.258138,0.272126,0.260427,0.259499,0.007206,11
9,0.964028,0.035898,0.105582,0.014433,0.0001,0.25,"(1, 1)","{'clf__estimator__alpha': 0.0001, 'vect__max_d...",0.295957,0.280264,0.287640,0.292218,0.280519,0.287320,0.006241,1


In [100]:
model4.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('vect', TfidfVectorizer()),
  ('clf', MultiOutputClassifier(estimator=SGDClassifier(n_jobs=-1)))],
 'estimator__verbose': False,
 'estimator__vect': TfidfVectorizer(),
 'estimator__clf': MultiOutputClassifier(estimator=SGDClassifier(n_jobs=-1)),
 'estimator__vect__analyzer': 'word',
 'estimator__vect__binary': False,
 'estimator__vect__decode_error': 'strict',
 'estimator__vect__dtype': numpy.float64,
 'estimator__vect__encoding': 'utf-8',
 'estimator__vect__input': 'content',
 'estimator__vect__lowercase': True,
 'estimator__vect__max_df': 1.0,
 'estimator__vect__max_features': None,
 'estimator__vect__min_df': 1,
 'estimator__vect__ngram_range': (1, 1),
 'estimator__vect__norm': 'l2',
 'estimator__vect__preprocessor': None,
 'estimator__vect__smooth_idf': True,
 'estimator__vect__stop_words': None,
 'estimator__vect__strip_accents': None,
 'estimator__vect__sublinear_tf': False,
 'estimator__vect__to

### 9. Export your model as a pickle file

In [24]:
pickle.dump(model4, open('model4.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.